In [ ]:
from GoogleNews import GoogleNews
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
#from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer
#from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

In [7]:

googlenews = GoogleNews()

googlenews = GoogleNews(lang='en')

googlenews = GoogleNews(start='06/20/2021',end='07/20/2021') #date range to change data size
googlenews = GoogleNews(encode='utf-8')

googlenews.get_news('') #Enter keyword


In [18]:
#Data Exploration
#googlenews.results() #dict of results
#googlenews.get_texts() #gets titles
#googlenews.get_links() #gets links to articles
#googlenews.total_count() #count of hits

In [8]:
sia = SentimentIntensityAnalyzer()

NewsDataFrame=pd.DataFrame(columns=['Title','Compound'])


for x in range(len(googlenews.get_texts())):
    NewsDataFrame=NewsDataFrame.append(pd.DataFrame({'Title': googlenews.get_texts()[x], 'Compound': sia.polarity_scores(googlenews.get_texts()[x])['compound']}, index=[0]),ignore_index=True)

In [9]:
##Asumption of scale based on data exploration
conditionals = [
    (NewsDataFrame['Compound'] < -.3),
    (NewsDataFrame['Compound']  > -.3) & (NewsDataFrame['Compound']  < .3),
    (NewsDataFrame['Compound']  > .3)
]

responses =[-1,0,1]

NewsDataFrame['Compound_Norm'] = np.select(conditionals, responses)

In [10]:
from sklearn.model_selection import train_test_split
Title_set = NewsDataFrame['Title'].values
Compound_set = NewsDataFrame['Compound_Norm'].values
Title_set_train, Title_set_test, y_train, y_test = train_test_split(Title_set, Compound_set, test_size=0.2, random_state=10000)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix,classification_report
vectorizer = CountVectorizer(stop_words='english')
#vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(Title_set_train)
X_test = vectorizer.transform(Title_set_test)
classifier = LogisticRegression(solver='liblinear',  C=10.0, random_state=0, max_iter=10000)
classifier.fit(X_train, y_train)

accuracy = classifier.score(X_test, y_test)
print('Accuracy: ', accuracy)
print(classifier.fit(X_train, y_train))
#print(classifier.predict_proba(X_train))
#print(confusion_matrix(y_train, classifier.predict(X_train)))
print(classification_report(y_train, classifier.predict(X_train))) #F1 Scores
print(confusion_matrix(y_train, classifier.predict(X_train))) #Confusion Matrix